In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the dataset file (e.g., labels.csv)
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "vasukipatel/face-recognition-dataset",
    "labels.csv",  # Adjust this to the actual CSV or data file
    pandas_kwargs={"usecols": ["filename", "label"]}  # Adjust if needed
)

print("First 5 records:\n", df.head())


In [ ]:
# Update this path if needed
dataset_dir = "/kaggle/input/face-recognition-dataset"
image_size = (100, 100)

def load_images(df, base_dir, image_size=(100, 100)):
    images, labels = [], []
    for _, row in df.iterrows():
        img_path = os.path.join(base_dir, row['filename'])
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, image_size)
            images.append(img)
            labels.append(row['label'])
    return np.array(images), np.array(labels)

X, y = load_images(df, dataset_dir, image_size)
X = X.astype('float32') / 255.0
print(f"Loaded {len(X)} images.")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)  # Change index if you have multiple webcams

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (100, 100)) / 255.0
        face_input = np.expand_dims(face_resized, axis=0)

        pred = model.predict(face_input)[0][0]
        label = "Person" if pred > 0.5 else "Other"
        color = (0, 255, 0) if pred > 0.5 else (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{label} ({pred:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
